In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/undersampledrenn/Repeated_Edited_Nearest_Neighbors_Under-sampled_Dataset.csv


In [21]:
import pandas as pd
import numpy as np
df = pd.read_csv('/kaggle/input/undersampledrenn/Repeated_Edited_Nearest_Neighbors_Under-sampled_Dataset.csv')
df = df[df['Label'] != 'BENIGN']

# Simple GRU With Startified K-Fold

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout
from keras.utils import np_utils

# Extract the features and label
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = np_utils.to_categorical(labels)

# Define the number of folds for stratified K-fold cross-validation
k_folds = 5

# Define the GRU model
model = Sequential()
model.add(GRU(units=32,  input_shape=(features.shape[1], 1), return_sequences=True))
model.add(GRU(units=32))
model.add(Dense(units=labels.shape[1], activation='softmax'))

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []
# Perform stratified K-fold cross-validation
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
for i, (train_index, test_index) in enumerate(skf.split(features, labels.argmax(1))):
    print("Fold", i+1)

    # Split the data into training and testing sets
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Reshape the data for RNN
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Train the model
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

    # Evaluate the model
    train_loss, train_accuracy = model.evaluate(X_train, y_train)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    
    train_acc_list.append(train_accuracy)
    train_loss_list.append(train_loss)
    test_acc_list.append(test_accuracy)
    test_loss_list.append(test_loss)

print("Average Training Accuracy: {}".format(np.mean(train_acc_list)))
print("Average Training Loss: {}".format(np.mean(train_loss_list)))
print("Average Testing Accuracy: {}".format(np.mean(test_acc_list)))
print("Average Testing Loss: {}".format(np.mean(test_loss_list)))

Fold 1
Epoch 1/5
5000/5000 [==============================] - 220s 43ms/step - loss: 0.4120 - accuracy: 0.8552 - val_loss: 0.2551 - val_accuracy: 0.9071
Epoch 2/5
5000/5000 [==============================] - 215s 43ms/step - loss: 0.2464 - accuracy: 0.9084 - val_loss: 0.2270 - val_accuracy: 0.9138
Epoch 3/5
5000/5000 [==============================] - 215s 43ms/step - loss: 0.2188 - accuracy: 0.9141 - val_loss: 0.2051 - val_accuracy: 0.9187
Epoch 4/5
5000/5000 [==============================] - 214s 43ms/step - loss: 0.2094 - accuracy: 0.9166 - val_loss: 0.1949 - val_accuracy: 0.9212
Epoch 5/5
1250/1250 [==============================] - 12s 10ms/step - loss: 0.2138 - accuracy: 0.9179
Fold 2
Epoch 1/5
5000/5000 [==============================] - 216s 43ms/step - loss: 0.1983 - accuracy: 0.9206 - val_loss: 0.1880 - val_accuracy: 0.9254
Epoch 2/5
5000/5000 [==============================] - 216s 43ms/step - loss: 0.1974 - accuracy: 0.9216 - val_loss: 0.1949 - val_accuracy: 0.9219
Epoch 3

# SIMPLE GRU

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, GRU

# Load the dataset
#df = pd.read_csv("data.csv")

# Split the dataset into features and labels
X = df.drop("Label", axis=1)
y = df["Label"]

features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Define the model architecture
model = Sequential()
model.add(GRU(units=32,  input_shape=(features.shape[1], 1), return_sequences=True))
model.add(GRU(units=32))
model.add(Dense(units=y.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Optuna

# This Structure is Following from a Paper

In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout
from keras.utils import np_utils

# Extract the features and label
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = np_utils.to_categorical(labels)

# Define the number of folds for stratified K-fold cross-validation
k_folds = 5

# Define the GRU model
model = Sequential()
model.add(GRU(units=32, input_shape=(features.shape[1], 1), return_sequences=True))
model.add(GRU(units=32))
model.add(LayerNormalization())
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(units=labels.shape[1], activation='softmax'))

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []
# Perform stratified K-fold cross-validation
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
for i, (train_index, test_index) in enumerate(skf.split(features, labels.argmax(1))):
    print("Fold", i+1)

    # Split the data into training and testing sets
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Reshape the data for RNN
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Train the model
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

    # Evaluate the model
    train_loss, train_accuracy = model.evaluate(X_train, y_train)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    
    train_acc_list.append(train_accuracy)
    train_loss_list.append(train_loss)
    test_acc_list.append(test_accuracy)
    test_loss_list.append(test_loss)

print("Average Training Accuracy: {}".format(np.mean(train_acc_list)))
print("Average Training Loss: {}".format(np.mean(train_loss_list)))
print("Average Testing Accuracy: {}".format(np.mean(test_acc_list)))
print("Average Testing Loss: {}".format(np.mean(test_loss_list)))

Fold 1
Epoch 1/5
5000/5000 [==============================] - 275s 45ms/step - loss: 0.4453 - accuracy: 0.8340 - val_loss: 0.2684 - val_accuracy: 0.8984
Epoch 2/5
5000/5000 [==============================] - 222s 44ms/step - loss: 0.2497 - accuracy: 0.9031 - val_loss: 0.2239 - val_accuracy: 0.9145
Epoch 3/5
5000/5000 [==============================] - 221s 44ms/step - loss: 0.2177 - accuracy: 0.9130 - val_loss: 0.2393 - val_accuracy: 0.9069
Epoch 4/5
5000/5000 [==============================] - 220s 44ms/step - loss: 0.1890 - accuracy: 0.9222 - val_loss: 0.1638 - val_accuracy: 0.9304
Epoch 5/5
1250/1250 [==============================] - 13s 10ms/step - loss: 0.1466 - accuracy: 0.9417
Fold 2
Epoch 1/5
5000/5000 [==============================] - 220s 44ms/step - loss: 0.1570 - accuracy: 0.9348 - val_loss: 0.1427 - val_accuracy: 0.9410
Epoch 2/5
5000/5000 [==============================] - 220s 44ms/step - loss: 0.1520 - accuracy: 0.9367 - val_loss: 0.1462 - val_accuracy: 0.9384
Epoch 3